In [3]:
import pandas as pd
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
import ast
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from joblib import dump, load
from imblearn.over_sampling import RandomOverSampler

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import joblib


In [4]:
encoder_genero = LabelEncoder()
encoder_dia = LabelEncoder()
encoder_establecimiento = OneHotEncoder(sparse=False)
encoder_ciudad = OneHotEncoder(sparse=False)
df = pd.read_excel('pre.xlsx')
df_ciudad = df[['user_id', 'ciudad']].copy().drop_duplicates().sort_values(by='user_id').dropna()
df['mean_monto'] = 0
df['numero_transacciones'] = 0
for index, row in df.iterrows():
    aux1 = df[(df['user_id']==row['user_id'])&(df['fecha']<=row['fecha'])].copy()
    df.at[index, 'mean_monto'] = aux1['monto'].mean()
    df.at[index, 'numero_transacciones'] = aux1.shape[0]
df['n_ciudad'] = df['user_id'].map(dict(zip(df_ciudad['user_id'], df_ciudad['ciudad'])))
df['ciudad'] = df['ciudad'].fillna(df['n_ciudad'])
df['ciudad'] = df['ciudad'].fillna(df_ciudad.ciudad.mode().values[0])
max_labels = df.groupby('user_id')['establecimiento'].value_counts().unstack(fill_value=0).idxmax(axis=1).to_frame().reset_index()
max_labels.columns = ['user_id', 'establecimiento']
df['n_establecimiento'] = df['user_id'].map(dict(zip(max_labels['user_id'], max_labels['establecimiento'])))
df['establecimiento'] = df['establecimiento'].fillna(df['n_establecimiento'])
df['establecimiento'] = df['establecimiento'].fillna(max_labels.establecimiento.mode().values[0])
df['dispositivo_n'] = df['dispositivo'].apply(ast.literal_eval)
df[['año', 'marca', 'proovedor']] = pd.json_normalize(df['dispositivo_n'])
df['género'] = encoder_genero.fit_transform(df['género'])
df['dia'] = encoder_dia.fit_transform(df['fecha'].dt.day_name())
encoded_establecimiento = encoder_establecimiento.fit_transform(df[['establecimiento']])
encoded_ciudad = encoder_ciudad.fit_transform(df[['ciudad']])
df['is_prime'] = df['is_prime'].map({True:1, False:0})
df['fraude'] = df['fraude'].map({True:1, False:0})
dump(encoder_genero, 'encoder_genero.joblib')
dump(encoder_dia, 'encoder_dia.joblib')
dump((encoder_establecimiento, encoder_establecimiento.categories_), 'encoder_establecimiento.joblib')
dump((encoder_ciudad, encoder_ciudad.categories_), 'encoder_ciudad.joblib')
df = pd.concat([df, pd.DataFrame(encoded_establecimiento, columns=encoder_establecimiento.categories_[0]),
                pd.DataFrame(encoded_ciudad, columns=encoder_ciudad.categories_[0])], axis=1)
df.drop(['user_id','transaction_id','dispositivo', 'status_txn','n_ciudad',
       'n_establecimiento', 'dispositivo_n', 'año', 'marca', 'proovedor', 'ciudad', 'establecimiento', 'fecha'], axis=1, inplace=True)
df

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,género,linea_tc,interes_tc,monto,hora,is_prime,dcto,cashback,fraude,mean_monto,...,Farmacia,Supermercado,Tienda departamental,Cancún,Ciudad de México,Guadalajara,Monterrey,Nezahualcóyotl,Tijuana,Toluca
0,0,31000,58,797.45,0,0,0.00,23.92,0,736.393333,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0,57000,36,1070.24,0,0,0.00,32.11,0,1070.240000,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0,61000,46,409.42,0,0,0.00,12.28,0,409.420000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0,79000,62,112.59,0,0,0.00,3.38,0,112.590000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0,42000,59,1.00,0,0,0.00,0.03,0,1.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26950,0,30000,38,701.61,23,0,0.00,21.05,0,438.170000,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
26951,1,47000,35,368.91,23,0,18.45,11.07,0,673.000000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
26952,1,77000,37,108.18,23,0,0.00,3.25,0,553.391111,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
26953,2,89000,54,1308.53,23,0,0.00,39.26,0,553.165455,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [5]:
X = df.drop('fraude', axis=1)
y = df['fraude']

In [6]:
oversampler = RandomOverSampler(sampling_strategy= 1, random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X, y)

In [7]:
df_resampled = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='target')], axis=1)
df_resampled

,género,linea_tc,interes_tc,monto,hora,is_prime,dcto,cashback,mean_monto,numero_transacciones,...,Supermercado,Tienda departamental,Cancún,Ciudad de México,Guadalajara,Monterrey,Nezahualcóyotl,Tijuana,Toluca,target
0,0,31000,58,797.45,0,0,0.0,23.92,736.393333,3,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
1,0,57000,36,1070.24,0,0,0.0,32.11,1070.240000,1,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
2,0,61000,46,409.42,0,0,0.0,12.28,409.420000,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
3,0,79000,62,112.59,0,0,0.0,3.38,112.590000,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
4,0,42000,59,1.00,0,0,0.0,0.03,1.000000,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52305,1,70000,55,768.32,11,0,0.0,23.05,768.320000,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1
52306,0,45000,34,669.25,1,0,0.0,33.46,570.765000,4,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
52307,1,76000,42,868.79,10,0,0.0,26.06,416.712692,26,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1
52308,0,41000,43,240.34,0,0,0.0,7.21,628.175000,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
# Aplicar la misma transformación a los datos de prueba
X_test_scaled = scaler.transform(X_test)
random_forest_classifier = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=10, random_state=42)

In [9]:
random_forest_classifier.fit(X_train_scaled, y_train)

RandomForestClassifier(max_depth=10, min_samples_split=10, random_state=42)

In [10]:
X_train.columns

Index(['género', 'linea_tc', 'interes_tc', 'monto', 'hora', 'is_prime', 'dcto',
       'cashback', 'mean_monto', 'numero_transacciones', 'dia', 'Abarrotes',
       'Compra en línea', 'Farmacia', 'Supermercado', 'Tienda departamental',
       'Cancún', 'Ciudad de México', 'Guadalajara', 'Monterrey',
       'Nezahualcóyotl', 'Tijuana', 'Toluca'],
      dtype='object')

In [11]:
accuracy = random_forest_classifier.score(X_test_scaled, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.888071114509654


In [12]:
svm_model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)

# Entrenar el modelo
svm_model.fit(X_train_scaled, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = svm_model.predict(X_test_scaled)

# Evaluar el modelo
accuracy = svm_model.score(y_pred, y_test)
print("Accuracy:", accuracy)

ValueError: Expected 2D array, got 1D array instead:
array=[1. 0. 0. ... 1. 0. 1.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
joblib.dump(random_forest_classifier, 'modelo_clasificador.pkl')
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [13]:
pd.read_excel('pre.xlsx').columns

Index(['transaction_id', 'user_id', 'género', 'linea_tc', 'interes_tc',
       'monto', 'fecha', 'hora', 'dispositivo', 'establecimiento', 'ciudad',
       'status_txn', 'is_prime', 'dcto', 'cashback', 'fraude'],
      dtype='object')